# Analyze Zero-shot

In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import TensorDataset, DataLoader

RANDOM_SEED=1

def set_random_seed_data(seed):
    RANDOM_SEED = seed

def lowercase(text):
    return text.lower()

def remove_nonaplhanumeric(text):
    text = re.sub('[^0-9a-zA-Z]+', ' ', text) 
    return text

def remove_unnecessary_char(text):
    text = re.sub('\n',' ',text) # Remove every '\n'
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))',' ',text) # Remove every URL
    text = re.sub('  +', ' ', text) # Remove extra spaces
    text = text.strip()
    return text

def preprocess_text(text):
    text = lowercase(text)
    text = remove_nonaplhanumeric(text)
    text = remove_unnecessary_char(text)
    return text

def load_data(PATH):
    test = pd.read_csv(PATH, sep='\t', header=None)
    test = test.rename(columns={0: "text", 1: "label"})
    test = test[test['label'] != 'neutral']
    test['label'] = test['label'].apply(lambda x: 1 if x=='positive' else 0)
    test['text'] = test['text'].apply(lambda x: preprocess_text(x))
    
    return test

In [3]:
test = load_data('../../../../../data/prosa/data_testing_full.tsv')
test.head()

,text,label
0,kemarin gue datang ke tempat makan baru yang a...,0
1,kayak nya sih gue tidak akan mau balik lagi ke...,0
2,kalau dipikir pikir sebenarnya tidak ada yang ...,0
3,ini pertama kalinya gua ke bank buat ngurusin ...,0
4,waktu sampai dengan gue pernah disuruh ibu lat...,0


# Zero-shot

In [5]:
data = pd.read_csv('result_prosa_yelp_XLM_R_B_10981_0.5_full.csv')
data['y_pred_int'] = data['y_pred'].apply(lambda x: int(x>=0.5))
data['text'] = test['text'].values
data.head()

,y_pred,y_true,y_pred_int,text
0,0.682132,0,1,kemarin gue datang ke tempat makan baru yang a...
1,0.678687,0,1,kayak nya sih gue tidak akan mau balik lagi ke...
2,0.686631,0,1,kalau dipikir pikir sebenarnya tidak ada yang ...
3,0.680322,0,1,ini pertama kalinya gua ke bank buat ngurusin ...
4,0.687011,0,1,waktu sampai dengan gue pernah disuruh ibu lat...


In [6]:
for i in range(0, data.shape[0]):
    if (data.loc[i].y_pred_int == data.loc[i].y_true):
        print("\ny_pred: {}".format(data.loc[i].y_pred))
        print("y_true: {}".format(data.loc[i].y_true))        
        print("text: {}".format(data.loc[i].text))      


y_pred: 0.6929088999999999
y_true: 1
text: apl ini sangat membantu sekali dengan memberikan kemudahan dalam pesan tiket to bepergian ke mana pun sehingga saya tidak harus keluar lagi to mmbli tiket semoga aplikasi ini selalu lncar ya setiap x melakukan pemesanan

y_pred: 0.6935117
y_true: 1
text: dua kali sudah teman saya bookingkan tiket lewat aplikasi ini bagus bagus saja tidak sampai 2 mnit kode booking nya langsung dikirimkan mantap

y_pred: 0.6995606
y_true: 1
text: alhamdulillahirrabil allamin menggunakan aplikasi ini mudah paling suka adalah penukaran poin sebagai pembayaran tiket bisa dilakukan dengan mudah simpel dan terpercaya

y_pred: 0.6785876000000001
y_true: 1
text: sempat agak takut pesan tiket pesawat di sini karena ulasan yang lain tapi ingat teman pernah beli di sini lancar saja akhirnya memesan pas banget di tfansfer cuma hitungan detik e tiket dikirim terima kasih lo terima kasih promo nya lumayan jadi lumayan untung

y_pred: 0.71513397
y_true: 1
text: aplikasi yan